## 基于 InceptionV3 微调

使用框架中自带的 InceptionV3，截取中间的一部分当作自己的模型

In [8]:
from tensorflow.keras.layers import *
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import vgg16, vgg19, inception_v3
from tensorflow.keras.applications.vgg19 import preprocess_input


import matplotlib.pyplot as plt
import cv2
import numpy as np

## 加载模型

In [9]:
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

## 从模型的输出端开始添加功能层

In [10]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(200, activation='softmax')(x)

## 构建模型

In [11]:
model = Model(inputs=base_model.input, outputs=out)

## 检查模型中前几层的名字

In [13]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
    if i >= 10:
        break

0 input_2
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d


## 锁住钱249层。只训练后面的层

In [14]:
for layer in model.layers[:249]:
    layer.trainable = False
    
for layer in model.layers[249:]:
    layer.trainable = True

## 需要重新编译模型

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])